# Notebook with preprocessing collected data

In [1]:
import numpy as np
import re
import pandas as pd
import json

Данные были собраны с фильтрацией по количеству комнат, так что, объединяя собранные csv файлы, будем добавлять дополнительный столбец с количеством комнат.
Всего было собрано 6 датасетов (от 1-комн. квартир до 6-комн. квартир)

The data was collected with filtering by the number of rooms, so during concatenating the collected csv files, we will add an additional column with the number of rooms.
In total, 6 datasets were collected (from 1 room apartments up to 6 rooms apartments).

In [2]:
DATA_PATH = "./data/"

In [3]:
df_cian = pd.read_csv(DATA_PATH + "cian_houses_1_flats.csv")
df_cian["room_count"] = 1

for i in range(2, 6 + 1):
    df_tmp = pd.read_csv(DATA_PATH + f"cian_houses_{i}_flats.csv")
    df_tmp["room_count"] = i
    df_cian = pd.concat([df_cian, df_tmp], ignore_index=True, sort=False)

df_cian.head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.80253,37.620945,Марьина Роща,34,49,38.28,11.8,unknown,...,3.12,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1
1,49954800,"Москва, ЦАО, р-н Таганский, Славянская пл., 2/5с1",55.752541,37.635727,Китай,7,8,53.20,unknown,24.1,...,3.0,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301496936/,1
2,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,Филатов Луг,2,5,31.00,unknown,10.0,...,3.0,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1
3,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.4,unknown,...,3.01,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1
4,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.6,18.6,...,2.85,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1


In [4]:
df_cian.shape

(8053, 22)

Удалим дубликаты, если они есть

In [5]:
df_cian.drop_duplicates()

df_cian.shape

(8053, 22)

Разделим на численные и категориальные фитчи

In [6]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != 'object']
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == 'object']

In [7]:
numerical_features

['floor', 'floor_count', 'square', 'room_count']

В `numerical_features` должны присутствовать еще поля `price`, `geo_lat`, `geo_lng`, `living square`, `kitchen_square`, `ceiling_height`.


Если с последними 3-я полями понятно, что могут присутствовать `unknown` значения, то с первыми 3-я надо разобраться.

Рассмотрим `price`

In [8]:
type(df_cian["price"][0]), type(df_cian["geo_lat"][0]), type(df_cian["geo_lng"][0]), 

(int, float, float)

тип данных &mdash; str, посмотрим м.б. присутствуют неизвестные значения

In [9]:
df_cian[df_cian["price"] == "unknown"]

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
2733,unknown,"Москва, НАО (Новомосковский), № 82 квартал, Но...",unknown,unknown,Филатов Луг,7,7,58.4,unknown,11.4,...,2.7,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,Класс,https://www.cian.ru/sale/flat/296320615/,2
2734,unknown,"Москва, НАО (Новомосковский), № 82 квартал, Но...",unknown,unknown,Филатов Луг,7,7,58.4,unknown,11.4,...,2.7,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,Класс,https://www.cian.ru/sale/flat/299551896/,2


Видно, что произошла ошибка при загрузке данных.

Можно и удалить эти данные, но так как присутствует множество остальных значений, попробуем восстановить пропуски в ручную.

In [10]:
# If you want to delete data with "unknown" price
# df_cian = df_cian.drop(df_cian[df_cian["price"] == "unknown"].index)

In [11]:
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/", "price"] = "16544720"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/", "geo_lat"] = "55.571953"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/", "geo_lng"] = "37.383118"

df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "price"] = "16650000"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "location"] = "Москва, НАО (Новомосковский), Внуковское поселение, ул. Самуила Маршака, 23"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "geo_lat"] = "55.634116"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "geo_lng"] = "37.320039"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "metro"] = "Рассказовка"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "floor"] = 17
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "floor_count"] = 17
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "square"] = 59.0
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "living_square"] = 32.0
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "kitchen_square"] = 13.
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "ceiling_height"] = 2.8
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "view"] = "На улицу и двор"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "heating"] = "Центральное"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "breakdown"] = "Нет"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "house_type"] = "Панельный"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "year"] = 2016
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "author"] = "unknown"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "accomodation_type"] = "Вторичка"

In [12]:
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/"]


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
2733,16544720,"Москва, НАО (Новомосковский), № 82 квартал, Но...",55.571953,37.383118,Филатов Луг,7,7,58.4,unknown,11.4,...,2.7,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,Класс,https://www.cian.ru/sale/flat/296320615/,2


In [13]:
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/"]

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
2734,16650000,"Москва, НАО (Новомосковский), Внуковское посел...",55.634116,37.320039,Рассказовка,17,17,59.0,32.0,13.0,...,2.8,На улицу и двор,Панельный,Центральное,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/299551896/,2


Исправим типы данных

In [14]:
df_cian["price"] = df_cian["price"].apply(lambda x: int(x))
df_cian["geo_lat"] = df_cian["geo_lat"].apply(lambda x: float(x))
df_cian["geo_lng"] = df_cian["geo_lng"].apply(lambda x: float(x))

Посмотрим на численные и категориальные фитчи снова

In [15]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != 'object']
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == 'object']

In [16]:
numerical_features

['price', 'geo_lat', 'geo_lng', 'floor', 'floor_count', 'square', 'room_count']

Численные фитчи выправились, в категориальных остались только те, в которых может присутствовать "unknown"

In [17]:
df_cian["metro"].unique()

array(['Марьина Роща', 'Китай', 'Филатов Луг', 'Алексеевская', 'Тульская',
       'Трикотажная', 'ЦСКА', 'Пыхтино', 'Толстопальцево',
       'Улица 1905 года', 'Автозаводская', 'Раменки', 'Бибирево',
       'Маяковская', 'Нижегородская', 'Белорусская', 'Шелепиха',
       'Фонвизинская', 'Новокузнецкая', 'Народное Ополчение',
       'Улица Скобелевская', 'Нахимовский проспект', 'unknown',
       'Сухаревская', 'Бауманская', 'ВДНХ', 'Серпуховская', 'Опалиха',
       'Деловой центр', 'Калужская', 'Павшино', 'Пролетарская',
       'Кунцевская', 'Мичуринский проспект', 'Беляево', 'Фили',
       'Молжаниново', 'Молодёжная', 'Давыдково', 'Медведково',
       'Железнодорожная', 'Воробьёвы горы', 'Павелецкая', 'Хорошёво',
       'Щёлковская', 'Москва', 'Новокосино', 'Третьяковская',
       'Дмитровская', 'Прокшино', 'Перово', 'Измайловская', 'Борисово',
       'Волгоградский проспект', 'Минская', 'Аминьевская', 'Крылатское',
       'Зеленоград', 'Варшавская', 'Воронцовская', 'Котельники',
     

Присутствуют значения:
- "unknown"
- содержащие количество минут до станции метро
- содержащие слово откроется \ закрыто
- неполные названия
- Названия разного формата

Попробуем восстановить недостающие данные и отформатровать имеющиеся.

Отталкиваться будем от координат.

Возьмем за основу код, представленный здесь:

https://ru.stackoverflow.com/questions/1300085/Помогите-написать-функцию-Найти-ближайшую-станцию-метро


In [18]:
from math import radians, cos, sin, asin


def distance_haversine(point_1: tuple, point_2: tuple):
    d_earth = 2.0 * 6372.8
    lat1, long1 = tuple(radians(c) for c in point_1)
    lat2, long2 = tuple(radians(c) for c in point_2)
    d = sin((lat2 - lat1) / 2.0) ** 2.0 + cos(lat1) * cos(lat2) * sin(
        (long2 - long1) / 2.0) ** 2.0
    return d_earth * asin(d ** 0.5)


def find_nearest(point_1: tuple, points: dict):
    dists = {p: distance_haversine(point_1, points[p]) for p in points}
    name, dist = min(dists.items(), key=lambda d: d[1])
    return {'name': name, 'distance': dist,
            'dist_coef': 3 if dist <= 1.0 else 2 if dist < 2.0 else 1}


Импортируем json со станциями

In [19]:
with open("./data/metro_coords.json", "r") as file:
    data_metro = json.load(file)

df_metro = []
for line in data_metro["lines"]:
    for station in line["stations"]:
        df_metro.append([station["name"], station["lat"], station["lng"]])

df_metro = pd.DataFrame(df_metro, columns=["metro", "lat", "lng"])
df_metro

,metro,lat,lng
0,Новокосино,55.745113,37.864052
1,Новогиреево,55.752237,37.814587
2,Перово,55.750980,37.784220
3,Шоссе Энтузиастов,55.758090,37.751703
4,Авиамоторная,55.751933,37.717444
...,...,...,...
442,Кокошкино,55.599722,37.171389
443,Санино,55.583889,37.138333
444,Крёкшино,55.577778,37.109722
445,Победа,55.566111,37.093056


Очистим таблицу от значений станций метро, которых нет в импортированном списке (сделаем их "unknown")

In [20]:
stations_not_in_dict = []

for i in range(df_cian["metro"].unique().shape[0]):
    st = df_cian["metro"].unique()[i]
    if st not in df_metro["metro"].unique():
        stations_not_in_dict.append(st)

stations_not_in_dict = np.array(stations_not_in_dict)
stations_not_in_dict

array(['Китай', 'Филатов Луг', 'unknown', 'Молодёжная', 'Воробьёвы горы',
       'Хорошёво', 'Щёлковская', 'Москва', 'Зеленоград', 'Новомосковская',
       'Юго', 'Петровский Парк', 'Семёновская', 'Хорошёвская',
       'Тёплый Стан', 'Реутов', 'Корниловская откроется', 'Люберцы',
       'Бульвар Дмитрия', 'Соколиная гора', 'Бутырская',
       'Ватутинки откроется', 'Бирюлёво откроется', 'Улица Академика',
       'Новые Черёмушки', 'Матвеевская', 'Петровско', 'Терехово 16 мин',
       'Терехово 18 мин', 'Рабочий посёлок', 'Парк Культуры', 'Тропарёво',
       'Терехово 17 мин', 'Терехово 19 мин', 'Серебряный Бор',
       'Площадь трёх', 'Терехово 23 мин', 'Улица Сергея',
       'Кедровая откроется', 'Филёвский парк', 'Терехово 21 мин',
       'Бульвар Адмирала', 'Терехово 5 мин', 'Терехово 8 мин',
       'Терехово 7 мин', 'Терехово 22 мин', 'Алма',
       'Звенигородская откроется', 'Бульвар Генерала', 'Коммунарка',
       'Карамышевская'], dtype='<U24')

In [21]:
def transform_station_to_unknown(station):
    return "unknown" if station in stations_not_in_dict else station

Количество уникальных семплов метро ДО и ПОСЛЕ очистки

In [22]:
df_cian["metro"].unique().shape, df_cian["metro"].apply(transform_station_to_unknown).unique().shape

((330,), (280,))

Чистим

In [23]:
df_cian["metro"] = df_cian["metro"].apply(transform_station_to_unknown)

Применим функцию `find_nearest`

In [24]:
df_cian["point"] = tuple(zip(df_cian["geo_lat"], df_cian["geo_lng"]))
df_cian.head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count,point
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.802530,37.620945,Марьина Роща,34,49,38.28,11.8,unknown,...,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1,"(55.8025300783107, 37.62094537252544)"
1,49954800,"Москва, ЦАО, р-н Таганский, Славянская пл., 2/5с1",55.752541,37.635727,unknown,7,8,53.20,unknown,24.1,...,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301496936/,1,"(55.752541, 37.635727)"
2,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,unknown,2,5,31.00,unknown,10.0,...,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1,"(55.552637, 37.337172)"
3,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.4,unknown,...,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1,"(55.810466, 37.624247)"
4,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.6,18.6,...,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1,"(55.70659736274, 37.6322846801478)"


In [25]:
metro_points = {
    p[0]: (p[1], p[2]) for p in df_metro.values
}

def find_nearest_metro(point):
    return find_nearest(point, metro_points)["name"]

In [26]:
df_cian.loc[df_cian["metro"] == "unknown", "metro"] = df_cian.loc[
    df_cian["metro"] == "unknown", "point"
].apply(find_nearest_metro)
df_cian

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count,point
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.802530,37.620945,Марьина Роща,34,49,38.28,11.8,unknown,...,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1,"(55.8025300783107, 37.62094537252544)"
1,49954800,"Москва, ЦАО, р-н Таганский, Славянская пл., 2/5с1",55.752541,37.635727,Китай-город,7,8,53.20,unknown,24.1,...,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301496936/,1,"(55.752541, 37.635727)"
2,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,Аэропорт Внуково,2,5,31.00,unknown,10.0,...,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1,"(55.552637, 37.337172)"
3,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.4,unknown,...,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1,"(55.810466, 37.624247)"
4,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.6,18.6,...,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1,"(55.70659736274, 37.6322846801478)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8048,987184440,"Москва, ЦАО, р-н Пресненский, Большая Никитска...",55.756153,37.606640,Арбатская,3,5,235.40,122.0,16.8,...,На улицу и двор,Монолитный,unknown,unknown,unknown,Новостройка,Intermark Real Estate,https://www.cian.ru/sale/flat/304373926/,6,"(55.756153, 37.60664)"
8049,186000000,"Москва, ЦАО, р-н Тверской, ул. Ильинка, 4",55.753625,37.625873,Площадь Революции,4,4,336.40,222.5,26.0,...,На улицу и двор,Кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/279420911/,6,"(55.753625, 37.625873)"
8050,2455020000,"Москва, ЦАО, р-н Тверской, 1-я Тверская-Ямская...",55.771088,37.594868,Маяковская,9,9,1067.40,unknown,unknown,...,Во двор,Монолитный,unknown,unknown,unknown,Новостройка / Апартаменты,VESPER,https://www.cian.ru/sale/flat/284599342/,6,"(55.7710883704405, 37.5948676266339)"
8051,1796450000,"Москва, ЦАО, р-н Хамовники, Пречистенская наб....",55.739732,37.605508,Кропоткинская,5,7,724.00,627.0,25.0,...,На улицу и двор,Кирпичный,Автономная котельная,Нет,unknown,Вторичка / Пентхаус,unknown,https://www.cian.ru/sale/flat/300812075/,6,"(55.739732, 37.605508)"


Дропнем ненужную строку points

In [27]:
df_cian = df_cian.drop(columns=["point"])

Исправлено.

### Посмотрим на квартиры с нереалистичной площадью

In [28]:
df_cian[(df_cian["square"] <= 10.0) & (df_cian["room_count"] == 1)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
719,2990000,"Москва, ВАО, р-н Северное Измайлово, Сиреневый...",55.803155,37.785009,Щелковская,2,6,9.6,unknown,2.0,...,unknown,unknown,Блочный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301136222/,1


In [29]:
df_cian[(df_cian["square"] <= 20.0) & (df_cian["room_count"] == 2)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


In [30]:
df_cian[(df_cian["square"] <= 30.0) & (df_cian["room_count"] == 3)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
3110,1650000,"Московская область, Пушкинский городской округ...",55.971053,37.932351,Выхино,3,5,29.0,unknown,10.0,...,2.65,Во двор,Кирпичный,Центральное,Нет,unknown,Вторичка,unknown,https://ivanteyevka.cian.ru/sale/flat/304682314/,3


In [31]:
df_cian[(df_cian["square"] <= 40.0) & (df_cian["room_count"] == 4)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


In [32]:
df_cian[(df_cian["square"] <= 50.0) & (df_cian["room_count"] == 5)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


In [33]:
df_cian[(df_cian["square"] <= 60.0) & (df_cian["room_count"] == 6)].sort_values(by="square")

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


## Save merged data

In [34]:
df_cian.to_csv(DATA_PATH + "cian_houses_dataset.csv", index=False)